[IOAI 2025 (Beijing, China), Individual Contest](https://ioai-official.org/china-2025)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IOAI-official/IOAI-2025/blob/main/Individual-Contest/Antique/Solution/Antique_Solution.ipynb)

# Antique Painting Authentication: Reference Solution

## Step 1: Train Your Model

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering
from collections import Counter
from sklearn.svm import SVC
import os

TRAIN_PATH = "./training_set/" # The address of trainig set

#DATA_PATH is the secret environment variable to point the address of the validation set and test set on the testing machine. 
#You cannot access this address locally.
if os.environ.get('DATA_PATH'):  
    DATA_PATH = os.environ.get("DATA_PATH") + "/" 
else:
    DATA_PATH = ""  # Fallback for local testing

train = pd.read_csv(TRAIN_PATH + "training_set.csv")

X = np.array(train.iloc[:,:5])
y = np.array(train.iloc[:,5])

labeled_mask = y != 0
unlabeled_mask = y == 0
X_labeled = X[labeled_mask]
y_labeled = y[labeled_mask]
X_unlabeled = X[unlabeled_mask]

n_clusters = 2
spectral = SpectralClustering(n_clusters=n_clusters, affinity='rbf', gamma=10, random_state=42)
cluster_labels = spectral.fit_predict(X) 

cluster_to_label = {}
for cluster in range(n_clusters):

    labeled_in_cluster = y_labeled[cluster_labels[labeled_mask] == cluster]

    if len(labeled_in_cluster) > 0:
        most_common_label = Counter(labeled_in_cluster).most_common(1)[0][0]
        cluster_to_label[cluster] = most_common_label

pseudo_labels = np.array([cluster_to_label[cluster] for cluster in cluster_labels])

svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(X, pseudo_labels)

## Step 2: Make Predictions on the Validation and Test Set

In [ ]:
testA = np.array(pd.read_csv(DATA_PATH + "validation_set.csv"))
testB = np.array(pd.read_csv(DATA_PATH + "test_set.csv"))

predA = svm.predict(testA)
predB = svm.predict(testB)

## Step 3: Generate `submission.zip` for Submission

In [ ]:
import zipfile
import os

submissionA = pd.DataFrame(predA)
submissionA.to_csv("./submissionA.csv", index=False, header=False)

submissionB = pd.DataFrame(predB)
submissionB.to_csv("./submissionB.csv", index=False, header=False)

files_to_zip = ['./submissionA.csv', './submissionB.csv']
zip_filename = 'submission.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        zipf.write(file, os.path.basename(file))

print(f'{zip_filename} is created successfully!')